## 账号登录

In [13]:
!pip install --upgrade didatodolist --index-url https://pypi.org/simple

DEPRECATION: Loading egg at /data/anaconda3/lib/python3.12/site-packages/edcc-0.2.1-py3.12.egg is deprecated. pip 25.1 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330


In [14]:
from dida import DidaClient
client = DidaClient(email="523018705@qq.com", password="X1i2e3y4u5")
token = client.token

In [15]:
tasks = client.tasks.get_this_week_tasks()
tasks

{'completed': [{'id': '67b61f4f596486bc59febada',
   'title': 'ppt',
   'content': '',
   'priority': 0,
   'status': 2,
   'startDate': '2025-02-21 16:00:00',
   'dueDate': '2025-02-21 16:00:00',
   'projectId': '4024428bac6d182ac68da936',
   'columnId': '',
   'tagDetails': [],
   'kind': 'TEXT',
   'isAllDay': True,
   'items': [],
   'progress': 0,
   'modifiedTime': '2025-02-22 23:53:36',
   'createdTime': '2025-02-19 18:13:35',
   'completedTime': '2025-02-22 23:53:36',
   'completedUserId': 1014518876,
   'isCompleted': False,
   'creator': 1014518876,
   'timeZone': 'Asia/Shanghai',
   'isFloating': False,
   'reminders': [],
   'exDate': [],
   'etag': 'p000ukp2',
   'deleted': 0,
   'attachments': [],
   'imgMode': 0,
   'sortOrder': 2339500248288},
  {'id': '67b61f44596486bc59febac0',
   'title': '模板',
   'content': '',
   'priority': 0,
   'status': 2,
   'startDate': '2025-02-21 16:00:00',
   'dueDate': '2025-02-21 16:00:00',
   'projectId': '4024428bac6d182ac68da936',
   

In [4]:

tasks = client.tasks.get_today_tasks()
tasks


{'completed': [], 'uncompleted': []}

In [2]:
import requests

def get_ticktick_token(email, password):
    login_url = "https://dida365.com/api/v2/user/signon?wc=true&remember=true"
    headers = {
        "Content-Type": "application/json",
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
    }
    payload = {
        "username": email,
        "password": password
    }
    
    response = requests.post(login_url, json=payload, headers=headers)
    if response.status_code == 200:
        token = response.cookies.get("t")
        return token
    else:
        raise Exception("登录失败，请检查账号密码")

# 替换为你的邮箱和密码
email = "523018705@qq.com"
password = "Xieyu120807!!!"
token = get_ticktick_token(email, password)
token

Exception: 登录失败，请检查账号密码

## 获取任务全部信息（包括项目、清单、任务，标签，排序等）

In [21]:
def get_all_tasks(token, filters=None):
    """
    获取所有任务，支持多条件筛选
    
    参数说明:
    - token: 登录令牌
    - filters: 筛选条件字典，支持以下字段：
        - status: 任务状态 (0-未完成, 2-已完成)
        - priority: 优先级 (0-5)
        - project_id: 项目ID
        - tag_names: 标签名称列表
        - start_date: 开始时间
        - due_date: 截止时间
        
    返回:
    - list: 任务列表
    """
    tasks_url = "https://api.dida365.com/api/v2/batch/check/0"
    headers = {
        "Cookie": f"t={token}",
        "Content-Type": "application/json",
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36"
    }
    
    try:
        # 获取所有数据
        response = requests.get(tasks_url, headers=headers)
        response.raise_for_status()
        data = response.json()
        
        # 获取任务列表
        tasks = data.get('syncTaskBean', {}).get('update', [])
        
        # 如果没有筛选条件，直接返回所有任务
        if not filters:
            return tasks
            
        # 应用筛选条件
        filtered_tasks = []
        for task in tasks:
            match = True
            
            # 状态筛选
            if 'status' in filters and task.get('status') != filters['status']:
                match = False
                
            # 优先级筛选
            if 'priority' in filters and task.get('priority') != filters['priority']:
                match = False
                
            # 项目筛选
            if 'project_id' in filters and task.get('projectId') != filters['project_id']:
                match = False
                
            # 标签筛选
            if 'tag_names' in filters:
                task_tags = set(task.get('tags', []))
                filter_tags = set(filters['tag_names'])
                if not (task_tags & filter_tags):  # 如果没有交集
                    match = False
                    
            # 开始时间筛选
            if 'start_date' in filters and task.get('startDate'):
                if task['startDate'] < filters['start_date']:
                    match = False
                    
            # 截止时间筛选
            if 'due_date' in filters and task.get('dueDate'):
                if task['dueDate'] > filters['due_date']:
                    match = False
            
            if match:
                filtered_tasks.append(task)
                
        return filtered_tasks
        
    except Exception as e:
        print(f"获取任务失败: {str(e)}")
        return []

def get_projects(token, filters=None):
    """
    获取项目列表，包含项目下的任务，支持多条件筛选
    
    参数说明:
    - token: 登录令牌
    - filters: 筛选条件字典，支持以下字段：
        - name: 项目名称
        - color: 项目颜色
        - group_id: 项目组ID
        - include_tasks: 是否包含任务（默认True）
        
    返回:
    - list: 项目列表，每个项目包含其任务列表
    """
    tasks_url = "https://api.dida365.com/api/v2/batch/check/0"
    headers = {
        "Cookie": f"t={token}",
        "Content-Type": "application/json",
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36"
    }
    
    try:
        # 获取所有数据
        response = requests.get(tasks_url, headers=headers)
        response.raise_for_status()
        data = response.json()
        
        # 获取项目列表
        projects = data.get('projectProfiles', [])
        tasks = data.get('syncTaskBean', {}).get('update', [])
        
        # 处理项目数据
        result = []
        for project in projects:
            match = True
            
            # 应用筛选条件
            if filters:
                # 名称筛选
                if 'name' in filters and project.get('name') != filters['name']:
                    match = False
                    
                # 颜色筛选
                if 'color' in filters and project.get('color') != filters['color']:
                    match = False
                    
                # 项目组筛选
                if 'group_id' in filters and project.get('groupId') != filters['group_id']:
                    match = False
            
            if match:
                project_data = project.copy()
                
                # 添加任务列表（如果需要）
                # 如果filters为None或include_tasks未指定，默认为True
                if not filters or filters.get('include_tasks', True):
                    project_tasks = [
                        task for task in tasks
                        if task.get('projectId') == project['id']
                    ]
                    project_data['tasks'] = project_tasks
                    
                result.append(project_data)
                
        return result
        
    except Exception as e:
        print(f"获取项目失败: {str(e)}")
        return []

def get_tags(token, filters=None):
    """
    获取标签列表，包含标签下的任务，支持多条件筛选
    
    参数说明:
    - token: 登录令牌
    - filters: 筛选条件字典，支持以下字段：
        - names: 标签名称列表
        - color: 标签颜色
        - include_tasks: 是否包含任务（默认True）
        
    返回:
    - list: 标签列表，每个标签包含其任务列表
    """
    tasks_url = "https://api.dida365.com/api/v2/batch/check/0"
    headers = {
        "Cookie": f"t={token}",
        "Content-Type": "application/json",
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36"
    }
    
    try:
        # 获取所有数据
        response = requests.get(tasks_url, headers=headers)
        response.raise_for_status()
        data = response.json()
        
        # 获取标签和任务列表
        tags = data.get('tags', [])
        tasks = data.get('syncTaskBean', {}).get('update', [])
        
        # 处理标签数据
        result = []
        for tag in tags:
            match = True
            
            # 应用筛选条件
            if filters:
                # 名称筛选
                if 'names' in filters and tag.get('name') not in filters['names']:
                    match = False
                    
                # 颜色筛选
                if 'color' in filters and tag.get('color') != filters['color']:
                    match = False
            
            if match:
                tag_data = tag.copy()
                
                # 添加任务列表（如果需要）
                # 如果filters为None或include_tasks未指定，默认为True
                if not filters or filters.get('include_tasks', True):
                    tag_tasks = [
                        task for task in tasks
                        if tag['name'] in task.get('tags', [])
                    ]
                    tag_data['tasks'] = tag_tasks
                    
                result.append(tag_data)
                
        return result
        
    except Exception as e:
        print(f"获取标签失败: {str(e)}")
        return []
    
# 1. 获取所有任务
all_tasks = get_all_tasks(token)

# 2. 获取特定状态的任务
completed_tasks = get_all_tasks(token, {'status': 2})

# 3. 获取高优先级且带特定标签的任务
important_tasks = get_all_tasks(token, {
    'priority': 5,
    'tag_names': ['重要', '紧急']
})

# 4. 获取所有项目及其任务
all_projects = get_projects(token)

# 5. 获取特定项目
specific_project = get_projects(token, {'name': '工作'})

# 6. 获取项目（不包含任务）
projects_only = get_projects(token, {'include_tasks': False})

# 7. 获取所有标签及其任务
all_tags = get_tags(token)

# 8. 获取特定标签
specific_tags = get_tags(token, {'names': ['重要', '紧急']})

# 9. 获取特定颜色的标签（不包含任务）
colored_tags = get_tags(token, {
    'color': '#FF0000',
    'include_tasks': False
})

## 任务增删改查

In [ ]:
# 定义创建任务的函数
def create_task(token, title, content="", startDate=None, dueDate=None, priority=0):
    """
    创建一个新的任务
    
    参数说明:
    - token: 登录令牌
    - title: 任务标题
    - content: 任务内容/备注
    - startDate: 开始时间 (ISO格式的字符串,如: "2023-12-25T00:00:00.000+0000")
    - dueDate: 截止时间 (ISO格式的字符串)
    - priority: 优先级 (0-最低, 1-低, 3-中, 5-高)
    """
    create_url = "https://api.dida365.com/api/v2/task"
    headers = {
        "Cookie": f"t={token}",
        "Content-Type": "application/json",
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36"
    }
    
    # 准备任务数据
    task_data = {
        "title": title,
        "content": content,
        "priority": priority
    }
    
    # 添加可选的时间参数
    if startDate:
        task_data["startDate"] = startDate
    if dueDate:
        task_data["dueDate"] = dueDate

    response = requests.post(create_url, json=task_data, headers=headers)
    if response.status_code == 200:
        return response.json()
    else:
        raise Exception(f"创建任务失败: {response.text}")

# 定义更新任务状态的函数
def update_task_status(token, task_id, status=2):
    """
    更新任务的状态
    
    参数说明:
    - token: 登录令牌
    - task_id: 任务ID
    - status: 任务状态 (0-未完成, 2-已完成)
    """
    update_url = f"https://api.dida365.com/api/v2/task/{task_id}"
    headers = {
        "Cookie": f"t={token}",
        "Content-Type": "application/json",
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36"
    }
    
    # 获取当前任务信息
    response = requests.get(update_url, headers=headers)
    if response.status_code != 200:
        raise Exception("获取任务信息失败")
    print(response.json())
    # 更新任务状态
    task_data = response.json()
    task_data["status"] = status
    
    response = requests.put(update_url, json=task_data, headers=headers)
    if response.status_code == 200:
        return response.json()
    else:
        raise Exception(f"更新任务状态失败: {response.text}")

# 定义删除任务的函数
def delete_task(token, task_id, project_id):
    """
    删除指定的任务
    
    参数说明:
    - token: 登录令牌
    - task_id: 要删除的任务ID
    - project_id: 任务所属的项目ID
    
    返回值:
    - bool: 删除是否成功
    """
    batch_url = "https://api.dida365.com/api/v2/batch/task"
    headers = {
        "Cookie": f"t={token}",
        "Content-Type": "application/json",
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36"
    }
    
    # 准备删除任务的数据
    delete_data = {
        "delete": [
            {
                "taskId": task_id,
                "projectId": project_id
            }
        ]
    }
    
    try:
        response = requests.post(batch_url, json=delete_data, headers=headers)
        if response.status_code == 200:
            return True
        else:
            error_msg = response.text if response.text else f"HTTP {response.status_code}"
            print(f"删除失败: {error_msg}")
            return False
    except requests.exceptions.RequestException as e:
        print(f"请求错误: {str(e)}")
        return False
    except Exception as e:
        print(f"删除任务时发生错误: {str(e)}")
        return False

# 示例使用
"""
# 创建新任务
new_task = create_task(
    token,
    title="测试任务",
    content="这是一个测试任务的描述",
    startDate="2023-12-25T00:00:00.000+0000",
    dueDate="2023-12-26T00:00:00.000+0000",
    priority=3
)

# 更新任务状态为已完成
updated_task = update_task_status(token, new_task['id'], status=2)

# 删除任务
is_deleted = delete_task(token, new_task['id'])
print(f"任务删除{'成功' if is_deleted else '失败'}")
"""
# new_task = create_task(
#     token,
#     title="测试任务1111",
#     content="这是一个测试任务的描述",
#     startDate="2023-12-25T00:00:00.000+0000",
#     dueDate="2023-12-26T00:00:00.000+0000",
#     priority=3
# )
# # updated_task = update_task_status(token, new_task['id'], status=2)
# delete_task(token, new_task['id'])


# 创建一个测试任务
new_task = create_task(
    token,
    title="测试任务1111",
    content="这是一个测试任务的描述",
    startDate="2023-12-25T00:00:00.000+0000",
    dueDate="2023-12-26T00:00:00.000+0000",
    priority=3
)

# 删除该任务
result = delete_task(token, new_task['id'],new_task['projectId'])
print(f"任务删除{'成功' if result else '失败'}")

## 项目增删改查

In [15]:
def create_project(token, name, color="#FFD324", group_id=None, sort_order=None):
    """
    创建一个新的项目/清单
    
    参数说明:
    - token: 登录令牌
    - name: 项目名称
    - color: 项目颜色 (可选，默认为黄色)
    - group_id: 所属分组ID (可选)
    - sort_order: 排序顺序 (可选)
    
    返回:
    - dict: 创建成功的项目信息
    """
    create_url = "https://api.dida365.com/api/v2/project"
    headers = {
        "Cookie": f"t={token}",
        "Content-Type": "application/json",
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36"
    }
    
    # 准备项目数据
    project_data = {
        "name": name,
        "color": color,
        "inAll": True,
        "kind": "TASK",
        "viewMode": "list"
    }
    
    # 添加可选参数
    if group_id:
        project_data["groupId"] = group_id
    if sort_order:
        project_data["sortOrder"] = sort_order
    
    try:
        response = requests.post(create_url, json=project_data, headers=headers)
        if response.status_code == 200:
            return response.json()
        else:
            raise Exception(f"创建项目失败: {response.text}")
    except Exception as e:
        print(f"创建项目时发生错误: {str(e)}")
        return None

def get_project(token, project_id):
    """
    获取项目详细信息
    
    参数说明:
    - token: 登录令牌
    - project_id: 项目ID
    
    返回:
    - dict: 项目详细信息
    """
    project_url = f"https://api.dida365.com/api/v2/project/{project_id}"
    headers = {
        "Cookie": f"t={token}",
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36"
    }
    
    try:
        response = requests.get(project_url, headers=headers)
        if response.status_code == 200:
            return response.json()
        else:
            raise Exception(f"获取项目信息失败: {response.text}")
    except Exception as e:
        print(f"获取项目信息时发生错误: {str(e)}")
        return None

def update_project(token, project_id, new_project, name=None, color=None, group_id=None):
    """
    更新项目信息
    
    参数说明:
    - token: 登录令牌
    - project_id: 项目ID
    - new_project: 当前项目信息
    - name: 新的项目名称 (可选)
    - color: 新的项目颜色 (可选)
    - group_id: 新的分组ID (可选)
    
    返回:
    - dict/bool: 更新后的项目信息或成功状态
    """
    update_url = f"https://api.dida365.com/api/v2/project/{project_id}"
    headers = {
        "Cookie": f"t={token}",
        "Content-Type": "application/json",
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36"
    }
    
    # 更新需要修改的字段
    if name:
        new_project["name"] = name
    if color:
        new_project["color"] = color
    if group_id:
        new_project["groupId"] = group_id
    
    try:
        response = requests.put(update_url, json=new_project, headers=headers)
        if response.status_code == 200:
            try:
                return response.json()  # 尝试解析JSON响应
            except ValueError:
                return True  # 如果没有JSON响应但状态码是200，返回True表示更新成功
        else:
            raise Exception(f"更新项目失败: {response.text}")
    except Exception as e:
        print(f"更新项目时发生错误: {str(e)}")
        return None

def delete_project(token, project_id):
    """
    删除指定的项目
    
    参数说明:
    - token: 登录令牌
    - project_id: 要删除的项目ID
    
    返回:
    - bool: 删除是否成功
    """
    delete_url = f"https://api.dida365.com/api/v2/project/{project_id}"
    headers = {
        "Cookie": f"t={token}",
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36"
    }
    
    try:
        response = requests.delete(delete_url, headers=headers)
        if response.status_code == 200:
            return True
        else:
            error_msg = response.text if response.text else f"HTTP {response.status_code}"
            print(f"删除失败: {error_msg}")
            return False
    except requests.exceptions.RequestException as e:
        print(f"请求错误: {str(e)}")
        return False
    except Exception as e:
        print(f"删除项目时发生错误: {str(e)}")
        return False

# 示例使用
"""
# 创建新项目
new_project = create_project(
    token,
    name="测试项目",
    color="#FFD324"
)

# 获取项目信息
project_info = get_project(token, new_project['id'])

# 更新项目
updated_project = update_project(
    token,
    project_id=new_project['id'],
    name="更新后的项目名称",
    color="#FF0000"
)

# 删除项目
is_deleted = delete_project(token, new_project['id'])
print(f"项目删除{'成功' if is_deleted else '失败'}")
"""

## TAG增删改查


In [ ]:
def get_all_data(token):
    """
    获取所有数据（包括标签、任务等）
    
    参数说明:
    - token: 登录令牌
    
    返回:
    - dict: 所有数据
    """
    check_url = "https://api.dida365.com/api/v2/batch/check/0"
    headers = {
        "Cookie": f"t={token}",
        "Content-Type": "application/json",
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36"
    }
    
    try:
        response = requests.get(check_url, headers=headers)
        response.raise_for_status()
        return response.json()
    except Exception as e:
        print(f"获取数据失败: {str(e)}")
        return None

def create_tag(token, name, color="#FFD324"):
    """
    创建新标签
    
    参数说明:
    - token: 登录令牌
    - name: 标签名称
    - color: 标签颜色 (可选，默认黄色)
    
    返回:
    - dict: 创建的标签信息
    """
    batch_url = "https://api.dida365.com/api/v2/batch/tag"
    headers = {
        "Cookie": f"t={token}",
        "Content-Type": "application/json",
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36"
    }
    
    tag_data = {
        "add": [{
            "name": name,
            "label": name,
            "color": color,
            "sortOrder": -1099511693312,
            "sortType": "project",
            "parent": None,
            "type": 1  # 默认为个人标签
        }],
        "update": [],
        "delete": []
    }
    
    try:
        response = requests.post(batch_url, json=tag_data, headers=headers)
        if response.status_code == 200:
            return response.json()
        else:
            raise Exception(f"创建标签失败: {response.text}")
    except Exception as e:
        print(f"创建标签时发生错误: {str(e)}")
        return None

def rename_tag(token, old_name, new_name):
    """
    重命名标签
    
    参数说明:
    - token: 登录令牌
    - old_name: 原标签名称
    - new_name: 新标签名称
    
    返回:
    - bool: 重命名是否成功
    """
    rename_url = "https://api.dida365.com/api/v2/tag/rename"
    headers = {
        "Cookie": f"t={token}",
        "Content-Type": "application/json",
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36"
    }
    
    rename_data = {
        "name": old_name,
        "newName": new_name
    }
    
    try:
        response = requests.put(rename_url, json=rename_data, headers=headers)
        if response.status_code == 200:
            return True
        else:
            raise Exception(f"重命名标签失败: {response.text}")
    except Exception as e:
        print(f"重命名标签时发生错误: {str(e)}")
        return False

def merge_tags(token, source_tag_name, target_tag_name):
    """
    合并标签
    
    参数说明:
    - token: 登录令牌
    - source_tag_name: 源标签名称（将被合并的标签）
    - target_tag_name: 目标标签名称（合并后保留的标签）
    
    返回:
    - bool: 合并是否成功
    """
    merge_url = "https://api.dida365.com/api/v2/tag/merge"
    headers = {
        "Cookie": f"t={token}",
        "Content-Type": "application/json",
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36"
    }
    
    merge_data = {
        "fromName": source_tag_name,
        "toName": target_tag_name
    }
    
    try:
        response = requests.put(merge_url, json=merge_data, headers=headers)
        if response.status_code == 200:
            return True
        else:
            raise Exception(f"合并标签失败: {response.text}")
    except Exception as e:
        print(f"合并标签时发生错误: {str(e)}")
        return False

def update_tag_type(token, tag_name, is_shared=False):
    """
    更新标签类型（个人/共同标签）
    
    参数说明:
    - token: 登录令牌
    - tag_name: 标签名称
    - is_shared: 是否为共同标签
    
    返回:
    - bool: 更新是否成功
    """
    all_data = get_all_data(token)
    if not all_data:
        return False
    
    target_tag = None
    for tag in all_data.get('tags', []):
        if tag.get('name') == tag_name:
            target_tag = tag
            break
    
    if not target_tag:
        print(f"标签 '{tag_name}' 不存在")
        return False
    
    batch_url = "https://api.dida365.com/api/v2/batch/tag"
    headers = {
        "Cookie": f"t={token}",
        "Content-Type": "application/json",
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36"
    }
    
    update_data = {
        "add": [],
        "update": [{
            **target_tag,
            "type": 2 if is_shared else 1
        }],
        "delete": []
    }
    
    try:
        response = requests.post(batch_url, json=update_data, headers=headers)
        if response.status_code == 200:
            return True
        else:
            raise Exception(f"更新标签类型失败: {response.text}")
    except Exception as e:
        print(f"更新标签类型时发生错误: {str(e)}")
        return False

def delete_tag(token, tag_name):
    """
    删除标签
    
    参数说明:
    - token: 登录令牌
    - tag_name: 标签名称
    
    返回:
    - bool: 删除是否成功
    """
    all_data = get_all_data(token)
    if not all_data:
        return False
    
    target_tag = None
    for tag in all_data.get('tags', []):
        if tag.get('name') == tag_name:
            target_tag = tag
            break
    
    if not target_tag:
        print(f"标签 '{tag_name}' 不存在")
        return False
    
    batch_url = "https://api.dida365.com/api/v2/batch/tag"
    headers = {
        "Cookie": f"t={token}",
        "Content-Type": "application/json",
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36"
    }
    
    delete_data = {
        "add": [],
        "update": [],
        "delete": [target_tag['id']]
    }
    
    try:
        response = requests.post(batch_url, json=delete_data, headers=headers)
        if response.status_code == 200:
            return True
        else:
            raise Exception(f"删除标签失败: {response.text}")
    except Exception as e:
        print(f"删除标签时发生错误: {str(e)}")
        return False

# 使用示例
def tag_operations_example(token):
    """
    标签操作示例
    """
    try:
        # 1. 创建标签
        print("创建标签...")
        tag1 = create_tag(token, "工作", color="#FF0000")
        if tag1:
            print("- 创建标签'工作'成功")
        
        # 2. 创建另一个标签用于合并
        print("\n创建用于合并的标签...")
        tag2 = create_tag(token, "临时", color="#0000FF")
        if tag2:
            print("- 创建标签'临时'成功")
        
        # 3. 重命名标签
        print("\n重命名标签...")
        if rename_tag(token, "临时", "待处理"):
            print("- 重命名标签为'待处理'成功")
        
        # 4. 更新标签类型
        print("\n更新标签类型...")
        if update_tag_type(token, "工作", is_shared=True):
            print("- 将'工作'标签设置为共同标签成功")
        
        # 5. 合并标签
        print("\n合并标签...")
        if merge_tags(token, "待处理", "工作"):
            print("- 将'待处理'标签合并到'工作'标签成功")
        
        # 6. 最后清理
        print("\n清理标签...")
        if delete_tag(token, "工作"):
            print("- 删除标签'工作'成功")
        
    except Exception as e:
        print(f"\n操作过程中发生错误: {str(e)}")

# 运行示例
# tag_operations_example(token)

创建标签时发生错误: 创建标签失败: {"errorId":"cymao1lh@ctw6","errorCode":"unknown_exception","errorMessage":"Unknown exception","data":null}
